In [1]:
%pip install -q --disable-pip-version-check \
    evaluate==0.4.0 \
    py7zr==0.20.4 \
    sentencepiece==0.1.99 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.4.0 \
    trl==0.7.2 \
    bert_score
%pip install -q    wandb bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 2

## Set up all random seeds

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!wget https://github.com/wandb/edu/raw/main/llm-training-course/colab/utils.py

--2024-04-20 02:07:47--  https://github.com/wandb/edu/raw/main/llm-training-course/colab/utils.py
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wandb/edu/main/llm-training-course/colab/utils.py [following]
--2024-04-20 02:07:48--  https://raw.githubusercontent.com/wandb/edu/main/llm-training-course/colab/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8155 (8.0K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   7.96K  --.-KB/s    in 0s      

2024-04-20 02:07:48 (93.1 MB/s) - ‘utils.py’ saved [8155/8155]



In [5]:
from google.colab import output
output.enable_custom_widget_manager()

In [54]:
PROJECT = "FlanT5-Lora"
MODEL_NAME = 'google/flan-t5-base'
DATASET = "MeQSum"
WANDB_ID = "normal_seq2seq"

In [7]:
import wandb
wandb.init(project=PROJECT, # the project I am working on
           tags=[MODEL_NAME, DATASET],
           notes ="Fine tuning FlanT5 with MeQSum Dataset. Prompt Instruction",
           id=WANDB_ID, resume='allow') # the Hyperparameters I want to keep track of

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import random

# tqdm library makes the loops show a smart progress meter.
from tqdm import tqdm
tqdm.pandas()

In [9]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [10]:
TRAIN_PATH = '/content/drive/MyDrive/cs577_proj_dataset/train.jsonl'
VAL_PATH = '/content/drive/MyDrive/cs577_proj_dataset/validate.jsonl'
TEST_PATH = '/content/drive/MyDrive/cs577_proj_dataset/test.jsonl'

In [11]:
random_seed = 0
np.random.seed(random_seed)
torch.manual_seed(random_seed)
random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [12]:
from datasets import load_dataset

# Define the paths to your files

def create_hf_dataset(train_path, val_path, test_path):
    data_files = {}
    data_files["train"] = train_path
    data_files["test"]  = test_path
    data_files["validation"] = val_path

    # Load the datasets
    dataset = load_dataset('json', data_files=data_files)

    # Print the first example of the training dataset
    print(dataset)

    return dataset

# check the number of examples in each dataset

dataset = create_hf_dataset(
    TRAIN_PATH,
    VAL_PATH,
    TEST_PATH
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['idx', 'inputs', 'target'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['idx', 'inputs', 'target'],
        num_rows: 150
    })
    validation: Dataset({
        features: ['idx', 'inputs', 'target'],
        num_rows: 50
    })
})


In [13]:
dataset['train'].to_pandas()

,idx,inputs,target
0,0,SUBJECT: who and where to get cetirizine - D M...,Who manufactures cetirizine?
1,1,who makes bromocriptine i am wondering what co...,Who manufactures bromocriptine?
2,2,SUBJECT: nulytely MESSAGE: Hello can you tell ...,"Who makes nulytely, and where can I buy it?"
3,3,Williams' syndrome I would like to have my dau...,Where can I get genetic testing for william's ...
4,4,ClinicalTrials.gov - Question - general inform...,Where can I get genetic testing for multiple m...
...,...,...,...
995,995,SUBJECT: after surgery of ear drum still same ...,What are the treatments for perforated eardrum?
996,996,SUBJECT: ClinicalTrials.gov - Question - speci...,What are the treatments for Glycogen storage d...
997,997,MESSAGE: I have numbness/tingling in my lower ...,Where can I find information and treatment for...
998,998,SUBJECT: sleep apnea MESSAGE: I was diagnosed ...,How long does swelling from sleep apnea take t...


In [ ]:
# with wandb.init(project=PROJECT, id="trial_run_normal_seq2seq", job_type="dataset", resume="allow"):
#    wbtrain = wandb.Table(data=dataset['train'].to_pandas())
#    wbvalidation = wandb.Table(data=dataset['validation'].to_pandas())
#    wbtest = wandb.Table(data=dataset['test'].to_pandas())
#    wandb.log({"meqsum_train": wbtrain})
#    wandb.log({"meqsum_validation": wbvalidation})
#    wandb.log({"meqsum_test": wbtest})

In [26]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def print_number_of_trainable_model_parameters(model, tag="original_model"):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    with wandb.init(project=PROJECT, job_type="log_parameters", id=WANDB_ID, resume='allow'):
      wandb.log({f'{tag}': {"trainable_model_params":trainable_model_params}})
      wandb.log({f'{tag}': {"all_model_params":all_model_params}})
      wandb.log({f'{tag}': {"percentage_of_trainable_model_parameters": 100 * trainable_model_params}} )

    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params}%"

print(print_number_of_trainable_model_parameters(original_model))

In [16]:
# Define W&B Table to store generations
columns = ["index", "medical question", "prompt", "human_summary", "zero_shot_output"]
table = wandb.Table(columns=columns)

In [28]:
START_PROMPT = 'Summarize the following medical question using context around it.\n\n'
END_PROMPT = '\n\nQuestion Summary:'
NER_PROMPT = ''
NER_END_PRMPT = ''

In [29]:
def tokenize_function(example):
    prompt = [START_PROMPT + question + END_PROMPT for question in example["inputs"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["target"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['idx', 'inputs', 'target',])

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [ ]:
# tokenized_datasets['train'].to_pandas().head().iloc[0]['input_ids']

In [ ]:
# with wandb.init(project=PROJECT, job_type="dataset"):
#    wbtrain_tokenized = wandb.Table(data=tokenized_datasets['train'].to_pandas())
#    wbvalidation_tokenized = wandb.Table(data=tokenized_datasets['validation'].to_pandas())
#    wbtest_tokenized = wandb.Table(data=tokenized_datasets['test'].to_pandas())
#    wandb.log({"MeQSum_train_tokenized": wbtrain_tokenized})
#    wandb.log({"MeQSum_validation_tokenized": wbvalidation_tokenized})
#    wandb.log({"MeQSum_test_tokenized": wbtest_tokenized})

In [30]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (1000, 2)
Validation: (50, 2)
Test: (150, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 150
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 50
    })
})


In [31]:
from types import SimpleNamespace
from pathlib import Path
from tqdm.notebook import tqdm
from datetime import datetime
import nltk

In [ ]:
# from datasets import load_metric
# from bert_score import score


# # bert_score is not a part of the datasets library, so we'll use the bert_score package directly

# rouge = evaluate.load('rouge')
# bleu = evaluate.load("bleu")

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     print(predictions[0].shape)
#     print(labels.shape)
#     print('Starting to batch decode: ')
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     print('Batch Decoding finished: ')
#     # Split the predictions and labels into individual sentences
#     decoded_preds = [pred.strip() for pred in decoded_preds]
#     decoded_labels = [label.strip() for label in decoded_labels]

#     # print(decoded_preds)
#     # print(decoded_labels)

#     #print(type(inference_df['finetuned_model_summaries'].tolist()))

#     references_questions = [[target] for target in decoded_labels]
#     #prediction_questions = [[target] for target in inference_df['finetuned_model_summaries'].tolist()]
#     #print(references_questions)
#     # Compute BLEU score
#     bleu_results = bleu.compute(
#         predictions=decoded_preds,
#         references=references_questions
#     )

#     result = rouge.compute(predictions=decoded_preds, references=decoded_labels)["rougeL"]
#     bleu_result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
#     P, R, F1 = score(decoded_preds, decoded_labels, lang="en")

#     return {
#         "rougel_fmeasure": result,
#         "bleu": bleu_result["bleu"],
#         "bert_score_f1": F1.mean().item(),
#     }

In [32]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# config = SimpleNamespace(
#     # hyperparameters
#     learning_rate=1e-3,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     gradient_accumulation_steps=4,
#     num_train_epochs=100,
#     save_steps=5,
#     save_strategy='epoch', # we cannot set it to "no". Otherwise, the model cannot guess the best checkpoint.
#     eval_steps=5,
#     logging_steps=5,
#     evaluation_strategy='epoch',
#     warmup_steps=500,
#     save_total_limit=3,
#     load_best_model_at_end = True,
#     output_dir = f'./MeQSum-training-{str(int(time.time()))}'
# )


config = SimpleNamespace(
    # hyperparameters
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    num_train_epochs=50,
    save_steps=5,
    save_strategy='epoch', # we cannot set it to "no". Otherwise, the model cannot guess the best checkpoint.
    eval_steps=5,
    logging_steps=5,
    evaluation_strategy='epoch',
    warmup_steps=500,
    save_total_limit=2,
    load_best_model_at_end = True,
    output_dir = f'./MeQSum-training-{str(int(time.time()))}'
)

In [33]:
training_args = TrainingArguments(
    output_dir=config.output_dir,
    learning_rate=config.learning_rate,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
   num_train_epochs=config.num_train_epochs,
    save_steps=config.save_steps,
    save_strategy=config.save_strategy, # we cannot set it to "no". Otherwise, the model cannot guess the best checkpoint.
    eval_steps=config.eval_steps,
    logging_steps=config.logging_steps,
    evaluation_strategy=config.evaluation_strategy,
    warmup_steps=config.warmup_steps,
    save_total_limit=config.save_total_limit,
    load_best_model_at_end = config.load_best_model_at_end,
    report_to="wandb",
    run_name=f"Prompt_tuning_original_model-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [34]:
# free up GPU Memory
torch.cuda.empty_cache()

In [35]:
with wandb.init(project=PROJECT, id="normal_seq2seq", resume='allow'):
  trainer.train()

Epoch,Training Loss,Validation Loss
0,41.512500,43.029999
1,40.687500,43.029999
2,40.662500,42.869999
4,40.837500,42.610001
5,40.500000,42.450001
6,40.175000,42.230000
8,39.400000,41.230000
9,39.100000,40.810001
10,38.512500,40.349998
12,37.850000,39.310001


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


eval/loss,██████▇▇▇▆▆▆▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▁▅▆▄▃▅▅▅▃▃▂▅▄▅▃▅▅▄▁▃▇▂▆▅▂▃▆▄█▂▄▄▃▅▆▃▇▃▆▅
eval/samples_per_second,█▄▃▅▆▄▄▄▆▆▇▄▅▄▆▄▄▅█▆▂▇▃▄▇▆▃▅▁▇▅▅▆▄▃▆▂▆▃▄
eval/steps_per_second,█▄▃▅▆▄▄▄▆▆▇▄▅▄▆▄▄▅█▆▂▇▃▄▇▆▃▅▁▇▅▅▆▄▃▆▂▆▃▄
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▇█▇▇▇▇▇▆▆▅▅▅▅▄▃▃▂▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▁▂▂▃▃▄▄▅▆▆▇▇███▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▁▁
train/loss,██████▇▇▇▆▆▆▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁


In [36]:
trainer.state.best_model_checkpoint

'./MeQSum-training-1713579339/checkpoint-1281'

In [37]:
trainer.state.best_model_checkpoint
!mkdir flan-dialogue-summary-checkpoint
custom_path = "/content/drive/MyDrive/cs577_proj_dataset/MODELS/flan-MeQSum-normal_train-checkpoint_lr_1e_5/"
trainer.save_model(output_dir=custom_path)

In [ ]:
!ls -al ./flan-dialogue-summary-checkpoint/

In [ ]:
# with wandb.init(project=PROJECT, job_type="models"):
#   artifact = wandb.Artifact("instruct_model", type="model")
#   artifact.add_dir(custom_path)
#   wandb.save(custom_path)
#   wandb.log_artifact(artifact)

In [38]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained(custom_path, torch_dtype=torch.bfloat16)

In [39]:
instruct_model = instruct_model.to("cuda")

## Evaluation of the model

In [46]:
from bert_score import score

In [47]:
def compute_rogue_metric(inference_df):
  rouge = evaluate.load('rouge')

  finetuned_model_results = rouge.compute(
    predictions=inference_df['finetuned_model_summaries'].tolist(),
    references=inference_df['human_baseline_summaries'].tolist(),
    use_aggregator=True,
    use_stemmer=True,
  )

  return finetuned_model_results

def compute_bleu_metric(inference_df):
  # Load the BLEU metric
  bleu = evaluate.load("bleu")
  #print(type(inference_df['finetuned_model_summaries'].tolist()))

  references_questions = [[target] for target in inference_df['human_baseline_summaries'].tolist()]
  #prediction_questions = [[target] for target in inference_df['finetuned_model_summaries'].tolist()]
  #print(references_questions)
  # Compute BLEU score
  bleu_results = bleu.compute(
      predictions=inference_df['finetuned_model_summaries'].tolist(),
      references=references_questions
  )

  # print('BLEU SCORE:')
  # print(bleu_results['score'])
  return bleu_results['bleu']

def compute_bert_score(inference_df):
  # Compute BERTScore
  P, R, F1 = score(
      inference_df['finetuned_model_summaries'].tolist(),
      inference_df['human_baseline_summaries'].tolist(), lang="en")

  # print('BERT SCORE:')
  # print('Precision:', P.mean().item())
  # print('Recall:', R.mean().item())
  # print('F1 Score:', F1.mean().item())

  return P.mean().item(), R.mean().item(), F1.mean().item()


In [40]:
def generate_test_set_inference(dataset, instruct_model):
  questions = dataset['test'][0:]['inputs']
  human_baseline_summaries = dataset['test'][0:]['target']

  finetuned_model_summaries = []

  for _, question in enumerate(tqdm(questions)):
      prompt = START_PROMPT + question + END_PROMPT
      input_ids = tokenizer(prompt, return_tensors="pt").input_ids
      input_ids = input_ids.to("cuda")

      instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
      original_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
      finetuned_model_summaries.append(original_model_text_output)

  zipped_summaries = list(zip(human_baseline_summaries,finetuned_model_summaries))

  df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries','finetuned_model_summaries'])

  print(df.head())

  return df


In [42]:
inference_df = generate_test_set_inference(dataset, instruct_model)

  0%|          | 0/150 [00:00<?, ?it/s]

                            human_baseline_summaries  \
0  How can i get rid of a lower lip birthmark per...   
1       Is Magnesium Silicofluoride safe for people?   
2                      Could RhoGAM damage the baby?   
3  Could hydroxychloroquine and methotrexate make...   
4  Is there a relationship between Gadolinium and...   

                           finetuned_model_summaries  
0        Is there a cure for my lower lip birthmark?  
1  Is it safe to use Magnesium silicofluoride to ...  
2  I am not a nurse or Dr or student... I am just...  
3  Ask your doctor about a prescription for hydro...  
4  I have 2 Genovia Labs test results years apart...  


In [51]:
inference_df.head()

,human_baseline_summaries,finetuned_model_summaries
0,How can i get rid of a lower lip birthmark per...,Is there a cure for my lower lip birthmark?
1,Is Magnesium Silicofluoride safe for people?,Is it safe to use Magnesium silicofluoride to ...
2,Could RhoGAM damage the baby?,I am not a nurse or Dr or student... I am just...
3,Could hydroxychloroquine and methotrexate make...,Ask your doctor about a prescription for hydro...
4,Is there a relationship between Gadolinium and...,I have 2 Genovia Labs test results years apart...


In [48]:
rogue_score = compute_rogue_metric(inference_df)
bleu_score = compute_bleu_metric(inference_df)
bert_score_precision, bert_score_recall, bert_score_f1  = compute_bert_score(inference_df)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [52]:
print(f'ROGUE SCORE: {rogue_score}')
print(f'BLEU SCORE: {bleu_score}')
print(f'BERT SCORE: PRECISION: {bert_score_precision}, RECALL: {bert_score_recall}, F1: {bert_score_f1}')

ROGUE SCORE: {'rouge1': 0.2047785602152984, 'rouge2': 0.07386379276997923, 'rougeL': 0.1809481283224303, 'rougeLsum': 0.1817727500470298}
BLEU SCORE: 0.01960819807088865
BERT SCORE: PRECISION: 0.8663658499717712, RECALL: 0.8747613430023193, F1: 0.8700279593467712


In [ ]:
with wandb.init(project=PROJECT, id=WANDB_ID, resume="allow"):
  wandb.log({"rogue_score": rogue_score})
  wandb.log({"bleu_score": bleu_score})
  wandb.log({"bert_score_precision": bert_score_precision})
  wandb.log({"bert_score_recall": bert_score_recall})
  wandb.log({"bert_score_f1": bert_score_f1})


## Logging Test Set scores and inference to wandb

In [59]:
with wandb.init(project=PROJECT, id=WANDB_ID, job_type="dataset", resume="allow"):
   wbtest_inference = wandb.Table(data=inference_df)
   wandb.log({"meqsum_test_inference": wbtest_inference})

bert_score_f1,0.87003
bert_score_precision,0.86637
bert_score_recall,0.87476
bleu_score,0.01961
eval/loss,29.875
eval/runtime,1.2333
eval/samples_per_second,40.543
eval/steps_per_second,5.676
train/epoch,49.6
train/global_step,1550
train/grad_norm,180
